In [1]:
import pandas as pd

In [2]:
# If notebook is run by pipline it has parameters and if one wants to run it manually from here default values are given, so that he can

# Define default values for manual execution
input_directory = "../data/raw_data/"
output_directory = "../data/Processed_data/"
years = [2018, 2019, 2020]

## Wczytanie danych bets

In [3]:
bets: dict[str: pd.DataFrame] = {}
for year in years:
    file_name = f"{input_directory}{year}.xlsx"   # noqa
    bets[str(year)] = pd.read_excel(file_name)
    print(f"Loaded bets_{year} from {file_name}")

Loaded bets_2018 from ../data/raw_data/2018.xlsx
Loaded bets_2019 from ../data/raw_data/2019.xlsx
Loaded bets_2020 from ../data/raw_data/2020.xlsx


In [4]:
for year in bets:
    bets[year].loc[bets[year]["Tournament"] == "Adelaide International 1", "Location"] = "Adelaide 1"
    bets[year].loc[bets[year]["Tournament"] == "Adelaide International 2", "Location"] = "Adelaide 2"
    bets[year].loc[bets[year]["Tournament"] == "Western & Southern Financial Group Masters", "Location"] = "Cincinnati"
    bets[year].loc[bets[year]["Tournament"] == "bett1HULKS Indoors", "Location"] = "Cologne 1"
    bets[year].loc[bets[year]["Tournament"] == "bett1HULKS Championship", "Location"] = "Cologne 2"
    bets[year]["Location"] = bets[year]["Location"].replace({'Dubai ': 'Dubai', 'Belgrade ': 'Belgrade', 'Napoli':'Naples', 'Shenzhen ': 'Shenzhen', 'St Petersburg': 'St. Petersburg'})
    bets[year] = bets[year][~bets[year]['Location'].isin(['Turin'])]
    bets[year]['Loser'] = bets[year]['Loser'].replace({"Varillas J. P.": "Varillas J.P.", "Tseng C. H.": "Tseng C.H."})
    bets[year]['Winner'] = bets[year]['Winner'].replace({"Varillas J. P.": "Varillas J.P.", "Tseng C. H.": "Tseng C.H."})
    bets[year] = bets[year][~bets[year]['Tournament'].isin(['United Cup', 'Tour Finals', 'Masters Cup', 'NextGen Finals','Tokyo Olympics','Atp Cup', 'Great Ocean Road Open', 'Murray River Open',
     'Laver Cup', 'Melbourne Summer Set'])]
    bets[year].loc[bets[year]['Tournament']== 'BNP Paribas Masters', 'Location']='Paris 2'
    bets[year].loc[bets[year]['Tournament']== 'Belgrade Open', 'Location']='Belgrade 2'

### W ramce danych z 2019 roku jest błędnie wpisany zawodnik wraz z rankingiem. Poprawiamy ręcznie błąd

In [5]:
bets["2019"].loc[bets["2019"]['Loser'] == 'Monteiro J.', 'LRank'] = 99
bets["2019"].loc[bets["2019"]['Loser'] == 'Monteiro J.', 'LPts'] = 580.0
bets["2019"].loc[bets["2019"]['Loser'] == 'Monteiro J.', 'Loser'] = 'Monteiro T.'

### W 2020 roku w ramce bets nazwa 2 turniejów jest ze sobą błednie zamieniona. Poprawiamy ten błąd zamieniając nazwy turniejów na poprawne.

In [6]:
auckland_mask = (bets["2020"]['Location'] == 'Auckland') & (bets["2020"]['Tournament'] == 'ASB Classic')
adelaide_mask = (bets["2020"]['Location'] == 'Adelaide') & (bets["2020"]['Tournament'] == 'Adelaide International')

bets["2020"].loc[auckland_mask, ['Location', 'Tournament']] = ['Adelaide', 'Adelaide International']
bets["2020"].loc[adelaide_mask, ['Location', 'Tournament']] = ['Auckland', 'ASB Classic']

In [7]:
for year in bets:
    bets[year].rename(columns={'Location': 'tourney_location'}, inplace=True)
    bets[year].reset_index(drop=True, inplace=True)

In [8]:
for year in bets:
    print(f"Year {year}: {bets[year].columns}")

Year 2018: Index(['ATP', 'tourney_location', 'Tournament', 'Date', 'Series', 'Court',
       'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank',
       'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5',
       'L5', 'Wsets', 'Lsets', 'Comment', 'B365W', 'B365L', 'EXW', 'EXL',
       'LBW', 'LBL', 'PSW', 'PSL', 'MaxW', 'MaxL', 'AvgW', 'AvgL'],
      dtype='object')
Year 2019: Index(['ATP', 'tourney_location', 'Tournament', 'Date', 'Series', 'Court',
       'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank',
       'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5',
       'L5', 'Wsets', 'Lsets', 'Comment', 'B365W', 'B365L', 'PSW', 'PSL',
       'MaxW', 'MaxL', 'AvgW', 'AvgL'],
      dtype='object')
Year 2020: Index(['ATP', 'tourney_location', 'Tournament', 'Date', 'Series', 'Court',
       'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank',
       'WPts', 'LPts', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4'

## Rok 2018 ma dodatkowych bukmacherów których usuniemy dla zgodności

In [9]:
bets["2018"] = bets["2018"].drop(columns=["EXW", "EXL", "LBW", "LBL"])    

### Sprawdzamy czy teraz te same kolumny

In [10]:
from test_helpers.preprocessing import check_columns_same
check_columns_same(bets)

✅ Year 2019: Columns match with the first year (2018).
✅ Year 2020: Columns match with the first year (2018).


## Wczytanie danych github

In [11]:
github: dict[str: pd.DataFrame] = {}
for year in years:
    file_name = f"{input_directory}atp_matches_{year}.csv"
    github[str(year)] = pd.read_csv(file_name)
    print(f"Loaded github_{year} from {file_name}")

Loaded github_2018 from ../data/raw_data/atp_matches_2018.csv
Loaded github_2019 from ../data/raw_data/atp_matches_2019.csv
Loaded github_2020 from ../data/raw_data/atp_matches_2020.csv


## Sprawdzenie czy zbiory mają takie same kolumny

In [12]:
check_columns_same(github)

✅ Year 2019: Columns match with the first year (2018).
✅ Year 2020: Columns match with the first year (2018).


### Zgodnie z założeniami usuwamy Finals, NextGen Finals, United Cup oraz mecze Davis Cup

In [13]:
for year in github:
    github[year]=github[year][~github[year]['tourney_name'].str.contains('Davis Cup', na=False)]
    github[year] = github[year][~github[year]['tourney_name'].isin(['United Cup', 'Tour Finals', 'NextGen Finals','Tokyo Olympics','Atp Cup', 'Great Ocean Road Open', 'Murray River Open',
  'Laver Cup','Melbourne Summer Set', 'Melbourne'])]
    github[year]["tourney_name"] = github[year]["tourney_name"].replace({'Belgrade ': 'Belgrade'})

### W tej ramce kolumna 'tourney_name' to dla większości turniejów lokalizacja turnieju a nie jego nazwa, a potrzebujemy żeby nazwa turnieju była taka jak w ramce bets. Dla turniejów, dla których 'tourney_name' to nie lokalizacja, ręcznie wpisujemy lokalizację

In [14]:
for year in github:
    github[year].rename(columns={'tourney_name': 'tourney_location'}, inplace=True)
    github[year]['tourney_location'] = github[year]['tourney_location'].replace({"Australian Open": "Melbourne", "Indian Wells Masters": "Indian Wells", "Miami Masters": "Miami", "Monte Carlo Masters": "Monte Carlo", "Madrid Masters": "Madrid", "Rome Masters": "Rome", "Roland Garros": "Paris", "s Hertogenbosch": "'s-Hertogenbosch", "Queen's Club": "Queens Club", "Wimbledon": "London", "Cincinnati Masters": "Cincinnati", "Us Open": "New York", "US Open": "New York", "Astana": "Nur-Sultan", "Shanghai Masters": "Shanghai", "Paris Masters": "Paris 2", "Rio De Janeiro": "Rio de Janeiro", "Shenzen ":"Shenzen", "ATP Rio de Janeiro": "Rio de Janeiro",'St Petersburg': 'St. Petersburg'})

### Turniej Canada Masters jest w nieparzystych latach w Montrealu i w parzystych w Toronto

In [15]:
for year in github:
    if int(year) % 2 == 1:
        github[year]['tourney_location'] = github[year]['tourney_location'].replace({"Canada Masters": "Montreal"})
    else:
        github[year]['tourney_location'] = github[year]['tourney_location'].replace({"Canada Masters": "Toronto"})

### Sprawdzamy, że utworzona przez nas kolumnna 'tourney_location' odpowiada kolumnie 'tourney_location' w ramce bets

In [16]:
from test_helpers.preprocessing import compare_tourney_locations

compare_tourney_locations(github, bets, "github", "bets")

✅ 'tourney_location' column matches for year 2018.
✅ 'tourney_location' column matches for year 2019.
✅ 'tourney_location' column matches for year 2020.


### Potrzebujemy mieć takie same nazwy zawodników w obu ramkach, aby dodać do ramki bets id zawodnika z ramki github. Aktualnie w ramce bets nazwa zawodnika jest w formacie typu Djokovic N. a w ramce github w formacie typu Novak Djokovic, dlatego musimy stworzyć nazwy zawodników w skróconym w formacie w ramce github.

In [17]:
def transform_name(name: str):
    name_parts = name.split()
    first_name = name_parts[0]
    last_name = ' '.join(name_parts[1:])
    return f"{last_name} {first_name[0]}."

In [18]:
for year in github:
    github[year]['shortened_winner_name']=github[year]['winner_name'].apply(transform_name)
    github[year]['shortened_loser_name']=github[year]['loser_name'].apply(transform_name)

### Sprawdzamy czy skrócona nazwa zawodnika jednoznacznie określa zawodnika

In [19]:
from test_helpers.preprocessing import check_one_to_one_mapping

check_one_to_one_mapping(github, column_pairs = [('shortened_winner_name', 'winner_id'), ('shortened_loser_name', 'loser_id')], raise_error=False)

✅ Year 2018: One-to-one mapping between 'shortened_winner_name' and 'winner_id' is valid.

            ❌ One-to-one mapping violation between 'shortened_loser_name' and 'loser_id' in year 2018:
            - Unique 'shortened_loser_name': 290
            - Unique 'loser_id': 291
            This suggests that some 'shortened_loser_name' values map to multiple 'loser_id' values or vice versa.
            
✅ Year 2019: One-to-one mapping between 'shortened_winner_name' and 'winner_id' is valid.

            ❌ One-to-one mapping violation between 'shortened_loser_name' and 'loser_id' in year 2019:
            - Unique 'shortened_loser_name': 281
            - Unique 'loser_id': 282
            This suggests that some 'shortened_loser_name' values map to multiple 'loser_id' values or vice versa.
            
✅ Year 2020: One-to-one mapping between 'shortened_winner_name' and 'winner_id' is valid.
✅ Year 2020: One-to-one mapping between 'shortened_loser_name' and 'loser_id' is valid.


## Zobaczmy, którzy zawodnicy nie są jednoznacznie określeni przez skróconą nazwę

In [20]:
non_unique_losers={}
for year in github:
    loser_groups = github[year].groupby('shortened_loser_name')['loser_id'].nunique()
    non_unique_loser_names = loser_groups[loser_groups > 1].index.tolist()
    non_unique_losers[year] = github[year][github[year]['shortened_loser_name'].isin(non_unique_loser_names)][['shortened_loser_name', 'loser_id']].drop_duplicates()

for i in range(2018,2021):
    if not non_unique_losers[str(i)].empty:
        print(f"Lata {i} - Przegrani z niejednoznacznymi skróconymi nazwami:")
        print(non_unique_losers[str(i)])

Lata 2018 - Przegrani z niejednoznacznymi skróconymi nazwami:
     shortened_loser_name  loser_id
2532             Zhang Z.    111190
2633             Zhang Z.    105585
Lata 2019 - Przegrani z niejednoznacznymi skróconymi nazwami:
     shortened_loser_name  loser_id
2274             Zhang Z.    111190
2276             Zhang Z.    105585


## W celu rozróżnienia zawodników dopsujemy drugą literkę imienia

In [21]:
for year in github:
   github[year].loc[github[year]['loser_id'] == 111190, 'shortened_loser_name'] = 'Zhang Zh.'
   github[year].loc[github[year]['winner_id'] == 111190, 'shortened_winner_name'] = 'Zhang Zh.'
   github[year].loc[github[year]['loser_id'] == 105585, 'shortened_loser_name'] = 'Zhang Ze.'
   github[year].loc[github[year]['winner_id'] == 105585, 'shortened_winner_name'] = 'Zhang Ze.'

In [22]:
check_one_to_one_mapping(github, column_pairs = [('shortened_winner_name', 'winner_id'), ('shortened_loser_name', 'loser_id')], raise_error=True)

✅ Year 2018: One-to-one mapping between 'shortened_winner_name' and 'winner_id' is valid.
✅ Year 2018: One-to-one mapping between 'shortened_loser_name' and 'loser_id' is valid.
✅ Year 2019: One-to-one mapping between 'shortened_winner_name' and 'winner_id' is valid.
✅ Year 2019: One-to-one mapping between 'shortened_loser_name' and 'loser_id' is valid.
✅ Year 2020: One-to-one mapping between 'shortened_winner_name' and 'winner_id' is valid.
✅ Year 2020: One-to-one mapping between 'shortened_loser_name' and 'loser_id' is valid.


### Niektóre imiona i nazwiska ze względu na swoją unikalność lub brak konsekwencji w zapisie musimy zmodyfikować ręcznie

In [23]:
def replace_shortened_names(dataframes: [pd.DataFrame]):
    replacements = {
        "Varillas J.P.": "Varillas J.P.",
        "Meligeni Alves F.": "Meligeni Rodrigues F",
        "Arnaud Bailly G.": "Bailly G.",
        "Sung Nam J.": "Nam J.S.",
        "Chan Hong S.": "Hong S.",
        "Fa Rodriguez Taverna S.": "Rodriguez Taverna S.",
        "Pucinelli De Almeida M.": "Pucinelli de Almeida M.",
        "Alejandro Hernandez Serrano J.": "Hernandez A.",
        "Marcel Stebe C.": "Stebe C.M.",
        "Martin del Potro J.": "Del Potro J.M.",
        "Marco Moroni G.": "Moroni G.M.",
        "Tsonga J.": "Tsonga J.W.",
        "Ignacio Londero J.": "Londero J.I.",
        "Pablo Varillas J.": "Varillas J.P.",
        "Pablo Ficovich J.": "Ficovich J.P.",
        "C.H. Tseng": "Tseng C.H.",
        "Oconnell C.": "O Connell C.",
        "Elahi Galan D.": "Galan D.E.",
        "Auger Aliassime F.": "Auger-Aliassime F.",
        "Woo Kwon S.": "Kwon S.W.",
        "Barrios Vera T.": "Barrios M.",
        "Yunchaokete B.": "Bu Y.",
        "Manuel Cerundolo J.": "Cerundolo J.M.",
        "Martin Etcheverry T.": "Etcheverry T.",
        "Hugues Herbert P.": "Herbert P.H.",
        "Hsiou Hsu Y.": "Hsu Y.",
        "Andrea Huesler M.": "Huesler M.A.",
        "Kuznetsov A.": "Kuznetsov An.",
        "Son Kwiatkowski T.": "Kwiatkowski T.S.",
        "Li Z.": "Li Zh.",
        "Hsin Tseng C.": "Tseng C.H.",
        "Kumar Mukund S.": "Mukund S.",
        "Ramos A.": "Ramos-Vinolas A.",
        "J Wolf J.": "Wolf J.J.",
        "Zhang Ze": "Zhang Ze.",
        "Lin Wu T.": "Wu T.L.",
        "Hans Rehberg M.": "Rehberg M.",
        "Mpetshi Perricard G.": "Mpetshi G.",
        "Agustin Tirante T.": "Tirante T.A.",
        "Alberto Olivieri G.": "Olivieri G.",
        "Nicolae Madaras D.": "Madaras D.",
        "Cong Mo Y.": "Mo Y.",
        "Patrick Smith J.": "Smith J.P.",
        "Hsun Lu Y.":"Lu Y.H.",
        "Barrios M.":"Barrios Vera M.T.",
        "Lennard Struff J.":"Struff J.L.",
        "Shannan Zayid M.":"Zayid M.S.",
        "Garcia Lopez G.":"Garcia-Lopez G.",
        "Rubin Statham J.":"Statham J.",
        "Li Zh.":"Li Z.",
        "Menendez Maceiras A.":"Menendez-Maceiras A.",
        "Mingjie Lin J.":"Lin J.M.",
        "Aragone J.":"Aragone JC",
        "Moroni G.M.":"Moroni G.",
        "Hee Lee D.":"Lee D.H.",
        "Haider Maurer A.":"Haider-Maurer A.",
        "Estrella V.":"Estrella Burgos V.",
        "Ferreira Silva F.":"Silva F.F.",
        "Gimeno Traver D.":"Gimeno-Traver D.",
        "Samper Montana J.":"Samper-Montana J.",
        "Hernandez J.":"Hernandez-Fernandez J"
    }
    
    for df in dataframes:
        for old_name, new_name in replacements.items():
            df['shortened_winner_name'] = df['shortened_winner_name'].replace({old_name: new_name})
            df['shortened_loser_name'] = df['shortened_loser_name'].replace({old_name: new_name})

replace_shortened_names([github['2018'], github['2019'], github['2020']])


In [24]:
def replace_names(dataframes: [pd.DataFrame]):
    replacements = {
        "Tseng C.H.": "Tseng C.H.",
        "Varillas J.P.": "Varillas J.P.",
        "Meligeni Alves F.": "Meligeni Rodrigues F",
        "Zhang Ze":"Zhang Ze.",
        "O'Connell C.":"O Connell C.",
        "Galan D.": "Galan D.E.",
        "Aragone J.":"Aragone JC",
        "Dolgopolov O.":"Dolgopolov A.",
    }
    
    for df in dataframes:
        for old_name, new_name in replacements.items():
            df['Loser'] = df['Loser'].replace({old_name: new_name})
            df['Winner']= df['Winner'].replace({old_name:new_name})

replace_names([bets['2018'], bets['2019'], bets['2020']])

## Po poprawkach każdy zawodnik w ramce bets ma już id

In [25]:
for year in github:
    map_player_name_player_id = github[year].groupby('shortened_loser_name', as_index=False)[['shortened_loser_name', 'loser_id']].first()
    mapping = dict(zip(map_player_name_player_id['shortened_loser_name'], map_player_name_player_id['loser_id']))
    bets[year]['loser_id'] = bets[year]['Loser'].map(mapping)

In [26]:
for year in github:
    map_player_name_player_id = github[year].groupby('shortened_winner_name', as_index=False)[['shortened_winner_name', 'winner_id']].first()
    mapping = dict(zip(map_player_name_player_id['shortened_winner_name'], map_player_name_player_id['winner_id']))
    bets[year]['winner_id'] = bets[year]['Winner'].map(mapping)

In [27]:
from test_helpers.preprocessing import check_missing_player_ids

check_missing_player_ids(bets)

✅ Year 2018: No missing 'loser_id' or 'winner_id' values found.
✅ Year 2019: No missing 'loser_id' or 'winner_id' values found.
✅ Year 2020: No missing 'loser_id' or 'winner_id' values found.


### Skoro mamy już wspólne id zawodnika, oraz id turnieju to możemy w obu ramkach zdefiniować wspólne id meczu. Sprawdzamy czy id meczu jest unikalne dla każdego wiersza w ramkach

In [28]:
for year in github:
    github[year]['match_id'] = github[year]['tourney_location'].astype(str) + '_' + year + '_' + github[year]['winner_id'].astype(str) + '_' + github[year]['loser_id'].astype(str)
    bets[year]['match_id'] = bets[year]['tourney_location'].astype(str) + '_' + year + '_' +  bets[year]['winner_id'].astype(str) + '_' + bets[year]['loser_id'].astype(str)

In [29]:
from test_helpers.preprocessing import check_match_id_uniqueness

check_match_id_uniqueness(github, "github")
check_match_id_uniqueness( bets, "bets")

✅ Year 2018: All 'match_id' values are unique in github dataset.
✅ Year 2019: All 'match_id' values are unique in github dataset.
✅ Year 2020: All 'match_id' values are unique in github dataset.
✅ Year 2018: All 'match_id' values are unique in bets dataset.
✅ Year 2019: All 'match_id' values are unique in bets dataset.
✅ Year 2020: All 'match_id' values are unique in bets dataset.


### Sprawdźmy jeszcze czy sa jakieś mecze, które sa tylko w jednej ramce.

In [30]:
from test_helpers.preprocessing import check_match_id_consistency
check_match_id_consistency(bets, github, 'bets', 'github') 

✅ Year 2018: `match_id` values are consistent between bets and github.
✅ Year 2019: `match_id` values are consistent between bets and github.
✅ Year 2020: `match_id` values are consistent between bets and github.


In [31]:
for year in years:
    bets[str(year)].to_csv(output_directory +f'bets{year}.csv', index=False)   # noqa
    github[str(year)].to_csv(output_directory +f'github{year}.csv', index=False)